In [2]:
import sys, os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from src.generator import data_generator, clip_generator, label_generator
from src.util import characters, id_from_char, char_from_id
from src.export import display_progress

from pymongo import MongoClient
from tensorflow import keras

import numpy as np
import pickle

In [3]:
database_name = 'slippi'
collection_name = 'training_data' # first 100k clips are reserved for testing

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]
collection.estimated_document_count()

916647

In [6]:
cursor = collection.find()
N = collection.estimated_document_count()

In [7]:
from os.path import join, splitext
from random import random

for i, doc in enumerate(cursor):
    # get metadata / labels
    character = doc['character']
    code = doc['code']
    game_id = splitext(doc['game_id'])[0]
    clip_id = doc['clip_id']

    # construct filename
    filename = f'{clip_id}_{character}_{code}_{game_id}.pkl'
    
    if random() < .1:
        filepath = join(parentdir, 'data', 'test', filename)
    else:
        filepath = join(parentdir, 'data', 'train', filename)
        
    # unpickle istream
    doc['istream'] = pickle.loads(doc['istream'])

    # pickle whole document
    pickle.dump(doc, open(filepath, 'wb'))
    
    if i % 100 == 0:
        display_progress(i, N)
display_progress(N, N)

[####################] 916647 of 916647 - 100.0% 